Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
tf.__version__

'1.1.0'

In [3]:
pickle_file = '/home/victor/data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [34]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    #tf.reset_default_graph()
    
    # Input data.
    with tf.name_scope('data'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    with tf.variable_scope("Weights"):
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
        layer1_biases = tf.Variable(tf.zeros([depth]))
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
        layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        with tf.name_scope("Conv1"):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            tf.summary.histogram("Weights_1", layer1_weights)
            tf.summary.histogram("Biases_1", layer1_biases)
            tf.summary.histogram("Activations_1", conv)
            tf.summary.histogram("relu_1", hidden)
        with tf.name_scope("Conv2"):
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            tf.summary.histogram("Weights_2", layer2_weights)
            tf.summary.histogram("Biases_2", layer2_biases)
            tf.summary.histogram("Activations_2", conv)
            tf.summary.histogram("relu_2", hidden)
        with tf.name_scope("Reshape_1"):
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            tf.summary.histogram("Reshape_1", reshape)
        with tf.name_scope("Hidden_layer"):
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            tf.summary.histogram("Weights_3", layer3_weights)
            tf.summary.histogram("Biases_3", layer3_biases)
            tf.summary.histogram("Activations_hidden", hidden)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        tf.summary.scalar("CrossEntropy", loss)
    # Deve ser colocado aqui para execução normal do código, se colocado ao final fica muito lento
    summaries = tf.summary.merge_all()
    # Optimizer.
    with tf.name_scope("Train"):
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    # Predictions for the training, validation, and test data.
    #with tf.name_scope('accuracy'):
    train_prediction = tf.nn.softmax(logits)
    with tf.name_scope("Validation"):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    with tf.name_scope("Testing"):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [35]:
num_steps = 1001
tf.reset_default_graph()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard/1")
    writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summ, _, l, predictions = session.run(
          [summaries, optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.263568
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.482125
Minibatch accuracy: 43.8%
Validation accuracy: 51.1%
Minibatch loss at step 100: 1.381240
Minibatch accuracy: 50.0%
Validation accuracy: 68.2%
Test accuracy: 74.0%


In [59]:
num_steps = 101
tf.reset_default_graph()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard")
    #writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            summ = session.run([summaries], feed_dict=feed_dict)
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                   valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


AttributeError: 'list' object has no attribute 'value'

In [46]:
!tensorboard --logdir=/home/victor/tensorboard

TensorBoard 0.1.5 at http://lagex03:6006 (Press CTRL+C to quit) ^C



---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [21]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
   
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    with tf.name_scope("Layer-1_Weights"):
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    with tf.name_scope("Layer-1_Bias"):
        layer1_biases = tf.Variable(tf.zeros([depth]))
    with tf.name_scope("Layer-2_Weights"):
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    with tf.name_scope("Layer-2_Bias"):
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    with tf.name_scope("Layer-2_Weights"):
        layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    with tf.name_scope("Layer-3_Bias"):
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope("Layer-4_Weights"):
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    with tf.name_scope("Layer-4_Bias"):
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        # FIRST convolution. Changed the stride in convolution from 2 to 1
        with tf.name_scope("Conv-1"):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            tf.summary.histogram("Weights_1", layer1_weights)
            tf.summary.histogram("Biases_1", layer1_biases)
            tf.summary.histogram("Activations_1", hidden)
        # Max pooling of stride 2 and kernel size 2.
        with tf.name_scope("MaxPool-1"):
            pool = tf.nn.max_pool(hidden, [1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
            tf.summary.histogram("MaxPool_1", pool)
        # SECOND convolution. Changed the stride in convolution from 2 to 1
        with tf.name_scope("Conv-2"):
            conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            tf.summary.histogram("Weights_2", layer2_weights)
            tf.summary.histogram("Biases_2", layer2_biases)
            tf.summary.histogram("Activations_2", hidden)
        # Max pooling of stride 2 and kernel size 2.
        with tf.name_scope("MaxPool-2"):
            pool = tf.nn.max_pool(hidden,[1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
            tf.summary.histogram("MaxPool_2", pool)
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        with tf.name_scope("Hidden_layer-1"):
            pool = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            tf.summary.histogram("Weights_3", layer3_weights)
            tf.summary.histogram("Biases_3", layer3_biases)
            tf.summary.histogram("Activations_hidden-1", pool)
        with tf.name_scope("Output_layer-2"):
            hidden2 = tf.matmul(pool, layer4_weights) + layer4_biases
            tf.summary.histogram("Weights_3", layer4_weights)
            tf.summary.histogram("Biases_3", layer4_biases)
            tf.summary.histogram("Activations_hidden-2",hidden2)
        return hidden2

    # Training computation.
    logits = model(tf_train_dataset)
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        tf.summary.scalar("CrossEntropy", loss)
    # Deve ser colocado aqui para execução normal do código, se colocado ao final fica muito lento
    summaries = tf.summary.merge_all()
    # Optimizer.
    with tf.name_scope("Train"):
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    with tf.name_scope("Validation"):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    with tf.name_scope("Testing"):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [20]:
num_steps = 101
tf.reset_default_graph()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard/22/101steps")
    writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summ, _, l, predictions = session.run(
          [summaries, optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.453072
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.331065
Minibatch accuracy: 18.8%
Validation accuracy: 18.8%
Minibatch loss at step 100: 1.551825
Minibatch accuracy: 50.0%
Validation accuracy: 45.4%
Test accuracy: 49.4%


In [22]:
num_steps = 2001
tf.reset_default_graph()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard/Problem1_2-conv_1-hidden")
    writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summ, _, l, predictions = session.run(
          [summaries, optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.625011
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.647236
Minibatch accuracy: 50.0%
Validation accuracy: 50.0%
Minibatch loss at step 100: 1.195179
Minibatch accuracy: 50.0%
Validation accuracy: 63.5%
Minibatch loss at step 150: 1.392537
Minibatch accuracy: 62.5%
Validation accuracy: 73.6%
Minibatch loss at step 200: 0.614455
Minibatch accuracy: 75.0%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.410511
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.886875
Minibatch accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss at step 350: 1.039060
Minibatch accuracy: 62.5%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.397193
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Minibatch loss at step 450: 0.363421
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.310275
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

## Versão com Dropout adicionado

In [23]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    with tf.name_scope("Layer-1_Weights"):
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    with tf.name_scope("Layer-1_Bias"):
        layer1_biases = tf.Variable(tf.zeros([depth]))
    with tf.name_scope("Layer-2_Weights"):
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    with tf.name_scope("Layer-2_Bias"):
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    with tf.name_scope("Layer-2_Weights"):
        layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    with tf.name_scope("Layer-3_Bias"):
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope("Layer-4_Weights"):
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    with tf.name_scope("Layer-4_Bias"):
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data, keep_prob):
        # FIRST convolution. Changed the stride in convolution from 2 to 1
        with tf.name_scope("Conv-1"):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            tf.summary.histogram("Weights_1", layer1_weights)
            tf.summary.histogram("Biases_1", layer1_biases)
            tf.summary.histogram("Activations_1", hidden)
        # Max pooling of stride 2 and kernel size 2.
        with tf.name_scope("MaxPool-1"):
            pool = tf.nn.max_pool(hidden, [1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
            tf.summary.histogram("MaxPool_1", pool)
        # SECOND convolution. Changed the stride in convolution from 2 to 1
        with tf.name_scope("Conv-2"):
            conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            tf.summary.histogram("Weights_2", layer2_weights)
            tf.summary.histogram("Biases_2", layer2_biases)
            tf.summary.histogram("Activations_2", hidden)
        # Max pooling of stride 2 and kernel size 2.
        with tf.name_scope("MaxPool-2"):
            pool = tf.nn.max_pool(hidden,[1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
            tf.summary.histogram("MaxPool_2", pool)
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        with tf.name_scope("Hidden_layer-1"):
            pool = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            tf.summary.histogram("Weights_3", layer3_weights)
            tf.summary.histogram("Biases_3", layer3_biases)
            tf.summary.histogram("Activations_hidden-1", pool)
        with tf.name_scope("Dropout"):
            drop = tf.nn.dropout(pool, keep_prob)
            tf.summary.histogram("Dropout", drop)
        with tf.name_scope("Output_layer"):
            output = tf.matmul(drop, layer4_weights) + layer4_biases
            tf.summary.histogram("Weights_3", layer4_weights)
            tf.summary.histogram("Biases_3", layer4_biases)
            tf.summary.histogram("Activations_hidden-1", output)
        return output

    # Training computation.
    logits = model(tf_train_dataset, 0.5)
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        tf.summary.scalar("CrossEntropy", loss)
    summaries = tf.summary.merge_all()
    # Optimizer.
    with tf.name_scope("Train"):
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    with tf.name_scope("Validation"):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    with tf.name_scope("Testing"):
        test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [24]:
num_steps = 2001
tf.reset_default_graph()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard/Problem2_2-conv_1-hidden_dropout")
    writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summ, _, l, predictions = session.run(
          [summaries, optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.403714
Minibatch accuracy: 6.2%
Validation accuracy: 11.2%
Minibatch loss at step 50: 2.309096
Minibatch accuracy: 12.5%
Validation accuracy: 12.2%
Minibatch loss at step 100: 2.301245
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 150: 2.186071
Minibatch accuracy: 12.5%
Validation accuracy: 13.3%
Minibatch loss at step 200: 1.930157
Minibatch accuracy: 25.0%
Validation accuracy: 33.5%
Minibatch loss at step 250: 1.548060
Minibatch accuracy: 50.0%
Validation accuracy: 44.4%
Minibatch loss at step 300: 1.641595
Minibatch accuracy: 50.0%
Validation accuracy: 56.3%
Minibatch loss at step 350: 1.148065
Minibatch accuracy: 75.0%
Validation accuracy: 64.0%
Minibatch loss at step 400: 0.904979
Minibatch accuracy: 87.5%
Validation accuracy: 71.2%
Minibatch loss at step 450: 0.730177
Minibatch accuracy: 75.0%
Validation accuracy: 73.4%
Minibatch loss at step 500: 0.826016
Minibatch accuracy: 62.5%
Validation accuracy: 77.2%
Min

## Adição de uma camada fully conected ao final (parâmetros idênticos ao da rede anterior). 

In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    with tf.name_scope("Layer-1_Weights"):
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
    with tf.name_scope("Layer-1_Bias"):
        layer1_biases = tf.Variable(tf.zeros([depth]))
    with tf.name_scope("Layer-2_Weights"):
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
    with tf.name_scope("Layer-2_Bias"):
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    with tf.name_scope("Layer-2_Weights"):
        layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    with tf.name_scope("Layer-3_Bias"):
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope("Layer-4_Weights"):
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_hidden], stddev=0.1))
    with tf.name_scope("Layer-4_Bias"):
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope("Layer-5_Weights"):
        layer5_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
    with tf.name_scope("Layer-5_Bias"):
        layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, keep_prob):
        ##
        # FIRST convolution. Changed the stride in convolution from 2 to 1
        with tf.name_scope("Conv-1"):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            tf.summary.histogram("Weights_1", layer1_weights)
            tf.summary.histogram("Biases_1", layer1_biases)
            tf.summary.histogram("Activations_1", hidden)
        # Max pooling of stride 2 and kernel size 2.
        with tf.name_scope("MaxPool-1"):
            pool = tf.nn.max_pool(hidden, [1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
            tf.summary.histogram("MaxPool_1", pool)
        # SECOND convolution. Changed the stride in convolution from 2 to 1
        with tf.name_scope("Conv-2"):
            conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            tf.summary.histogram("Weights_2", layer2_weights)
            tf.summary.histogram("Biases_2", layer2_biases)
            tf.summary.histogram("Activations_2", hidden)
        # Max pooling of stride 2 and kernel size 2.
        with tf.name_scope("MaxPool-2"):
            pool = tf.nn.max_pool(hidden,[1, 2, 2, 1] , [1, 2, 2, 1], padding='SAME')
            tf.summary.histogram("MaxPool_2", pool)
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        with tf.name_scope("Hidden_layer-1"):
            pool = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            tf.summary.histogram("Weights_3", layer3_weights)
            tf.summary.histogram("Biases_3", layer3_biases)
            tf.summary.histogram("Activations_hidden-1", pool)
        with tf.name_scope("Dropout"):
            drop = tf.nn.dropout(pool, keep_prob)
            tf.summary.histogram("Dropout", drop)
        with tf.name_scope("Hidden_layer-2"):
            drop = tf.matmul(drop, layer4_weights) + layer4_biases
            tf.summary.histogram("Weights_3", layer4_weights)
            tf.summary.histogram("Biases_3", layer4_biases)
            tf.summary.histogram("Activations_hidden-1", drop)
        with tf.name_scope("Output_layer"):
            output = tf.matmul(drop, layer5_weights) + layer5_biases
            tf.summary.histogram("Weights_3", layer5_weights)
            tf.summary.histogram("Biases_3", layer5_biases)
            tf.summary.histogram("Activations_hidden-1", output)        
        return output
    # Training computation.
    logits = model(tf_train_dataset, 0.5)
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
        tf.summary.scalar("CrossEntropy", loss)
    summaries = tf.summary.merge_all()
    # Optimizer.
    with tf.name_scope("Train"):
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    with tf.name_scope("Validation"):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    with tf.name_scope("Testing"):
        test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

## O dobro de iterações

In [26]:
num_steps = 2001
tf.reset_default_graph()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter("/home/victor/tensorboard/Problem2_2-conv_2-hidden_dropout")
    writer.add_graph(session.graph)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summ, _, l, predictions = session.run(
          [summaries, optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            writer.add_summary(summ, global_step=step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.721958
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.307679
Minibatch accuracy: 25.0%
Validation accuracy: 19.7%
Minibatch loss at step 100: 2.028140
Minibatch accuracy: 18.8%
Validation accuracy: 52.3%
Minibatch loss at step 150: 1.657789
Minibatch accuracy: 37.5%
Validation accuracy: 63.5%
Minibatch loss at step 200: 1.159357
Minibatch accuracy: 62.5%
Validation accuracy: 63.3%
Minibatch loss at step 250: 0.835181
Minibatch accuracy: 75.0%
Validation accuracy: 71.8%
Minibatch loss at step 300: 1.440154
Minibatch accuracy: 56.2%
Validation accuracy: 67.1%
Minibatch loss at step 350: 1.344432
Minibatch accuracy: 62.5%
Validation accuracy: 76.5%
Minibatch loss at step 400: 0.817916
Minibatch accuracy: 75.0%
Validation accuracy: 75.8%
Minibatch loss at step 450: 0.565166
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500: 0.497335
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Mi